In [15]:
import cv2
import numpy as np
from scipy import ndimage
import random
import pyodbc

In [16]:
def read_files(directory):
    files = []
    for nm in os.listdir(directory):
        if file[-3:] == 'png':
            files.append(file)
    return files
def read_images(files, directory):
    images = [cv2.imread(directory + '\\' + file) for file in files]
    return np.array(images)

def horizontal_flip(images):
    flipped = [cv2.flip(img, 0) for img in images]
    return flipped

def vertical_flip(images):
    flipped = [cv2.flip(img, 1) for img in images]
    return flipped

def rotations(images):
    im_shape = images.shape
    rotated = np.array([[ndimage.rotate(img, (x + 1) * 90) for x in range(2)] for img in images])
    flattened = rotated.reshape(im_shape[0] * 3, images[1], images[2], images[3])
    return flattened

def noise(images):
    rand = np.random.randint(0, 50, tuple(images.shape))
    noised = images + rand
    return noised            
    
def prepare_data(training, labels):
    horz = horizontal_flip(training)
    vert = vertical_flip(training)
    noised = noise(training)
    rot = rotations(training)
    new_labels = np.concatenate((np.tile(labels, 4), np.repeat(labels, 3)))
    new_training = np.concatenate((training, horz, vert, noised, rot))
    return new_training, new_labels

In [18]:
def get_cursor():
    server = r'localhost\SQLEXPRESS'
    database = 'Patients'
    cnxn = pyodbc.connect('DRIVER={ODBC Driver 13 for SQL Server};SERVER='+server+';DATABASE='+database+';Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    cnxn.autocommit = True
    return cursor

In [26]:
def get_labels(files, cursor):
    labels = []
    for patient in files:
        cursor.execute("select GleasonGradePrimary, GleasonGradeSecondary from Patient where EntityID = '{}'".format(patient[0:12]))
        labels.append(cursor.fetchone())
    return labels

In [27]:
def data(directory):
    files = read_files(directory)
    pre_images = read_images(files, directory)
    cursor = get_cursor()
    pre_labels = get_labels(files, cursor)
    training, labels = preapare_data(pre_images, pre_labels)
    return training, labels